In [1]:
#parameters
using DifferentialEquations
using LaTeXStrings
using Plots; pyplot(); # here, pyplot is used as a back-end to the Plots API
using BenchmarkTools
using Polynomials
using LinearAlgebra
using Sundials
using Combinatorics
using Flux
using BSON: @load

ArgumentError: ArgumentError: Package PyPlot not found in current path:
- Run `import Pkg; Pkg.add("PyPlot")` to install the PyPlot package.


In [2]:
#time calculation
@benchmark sol_analytic = map(x -> hex_a(u_hex, par_hex_a, x), xspan_a) |> vec2vec
@benchmark sol_numeric_Cp_T_dep = hex_n_Cp_T_dep(u_hex, par_hex_n_Cp_T_dep, xspan_n)
@benchmark sol_numeric_Cp_T_dep = hex_n_Cp_T_indep(u_hex, par_hex_n_Cp_T_dep, xspan_n);


LoadError: UndefVarError: @benchmark not defined

In [3]:
#data driven model(linear regression time calculation)
# j = [1 to 12 orders]
function TimeCalc(j)
    Phi = [phi_m(Xscale[:,i];n=j) for i in 1:size(Xscale,2)]  |> x -> reduce(hcat,x)
    β= Yscale/Phi
    E1= Yscale-β*Phi
    e1 = norm(E1)/sqrt(length(Yscale))
end
t1 = @benchmark TimeCalc(1)
t2 = @benchmark TimeCalc(2)
t3 = @benchmark TimeCalc(3)
t4 = @benchmark TimeCalc(4)
t5 = @benchmark TimeCalc(5)
t6 = @benchmark TimeCalc(6)
t7 = @benchmark TimeCalc(7)
t8 = @benchmark TimeCalc(8)
t9 = @benchmark TimeCalc(9)
t10 = @benchmark TimeCalc(10)
t11 = @benchmark TimeCalc(11)
t12 = @benchmark TimeCalc(12)

LoadError: UndefVarError: @benchmark not defined

In [4]:
#hybrid model time calculation
#combining analytic model with regression model
function timecalc(k)
    function hex_a(inputs,par,β) 
    Twc, Tah, mdw, mda,UAx = inputs
    chpw, chpa = par
    # Stanton numbers for air and water.
    NSta = UAx/chpa/mda	# Stanton number for air, -
    NStw = UAx/chpw/mdw	# Stanton number for water, -
    NStd = NStw - NSta	# Difference in Stanton numbers, -
    #
    Tac = (NStd*Tah + NSta*(1-exp(-NStd))*Twc)/(NStw-NSta*exp(-NStd))    
    Tw(x) = ((NStw*exp(-NStd*x)-NSta)*Twc+(NStw - NStw*exp(-NStd*x))*Tac)/NStd
    Ta(x) = ((NSta*exp(-NStd*x)-NSta)*Twc+(NStw - NSta*exp(-NStd*x))*Tac)/NStd
    Phi = [phi_m(Xscale[:,i];n=norder) for i in 1:size(Xscale,2)]  |> x -> reduce(hcat,x)
    Ys=β*Phi
    return [Ys[1] Ys[end]]
    end
end    
@benchmark timecalc(k)

LoadError: UndefVarError: @benchmark not defined

In [5]:
#data driven model(non linear regression)
# time calculation
function TimeCalc(nz)
    #nz=node
    # defining model
    md = Chain(Dense(5,nz,tanh),Dense(nz,2))
    # Initial mapping
    Yd_0= md(Xd)
    #Defining loss function
    loss(x, y) = mean((md(x).-y).^2)
    #specyfying optimization method
    opt = ADAM(0.01, (0.99, 0.999))
    # Extracting parameters from the model
    par =Flux.params(md)
    # Training over nE epochs
    for i in 1:nE
        Flux.train!(loss,par,data,opt)
    end
    # Final mapping
    Yd_nE = md(Xd)
    # Calculating error
    RMSE = norm(Yd_nE-Yd)^2/sqrt(length(Yd)) 
end
t = @benchmark TimeCalc(nz);

LoadError: UndefVarError: @benchmark not defined

In [6]:
# hybrid model
#combining analytic model with non-regression model
function timecalc(k)
    function hex_a(inputs,par,md) # If x is an Array/tuple, the function can generate the temperature profile across x.
    Twc, Tah, mdw, mda,UAx, = inputs
    chpw, chpa = par
    # Stanton numbers for air and water.
    NSta = UAx/chpa/mda	# Stanton number for air, -
    NStw = UAx/chpw/mdw	# Stanton number for water, -
    NStd = NStw - NSta	# Difference in Stanton numbers, -
    #
    Tac = (NStd*Tah + NSta*(1-exp(-NStd))*Twc)/(NStw-NSta*exp(-NStd))    
    Tw(x) = ((NStw*exp(-NStd*x)-NSta)*Twc+(NStw - NStw*exp(-NStd*x))*Tac)/NStd
    Ta(x) = ((NSta*exp(-NStd*x)-NSta)*Twc+(NStw - NSta*exp(-NStd*x))*Tac)/NStd
    Yd_nE = md(Xd)
    return [ Yd_nE[1]  Yd_nE[end]]
    end
end
@benchmark timecalc(k);

LoadError: UndefVarError: @benchmark not defined